# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pyproj
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,272.96,88,95,5.43,GS,1731376041
1,1,peace river,56.2501,-117.2860,275.32,93,100,2.57,CA,1731376043
2,2,coquimbo,-29.9533,-71.3436,287.03,69,0,3.09,CL,1731375777
3,3,kamaishi,39.2667,141.8833,286.43,52,87,1.74,JP,1731376045
4,4,adamstown,-25.0660,-130.1015,295.02,64,100,1.27,PN,1731376046


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:

# Configure the map plot


# Display the map
#I kept recieving an error that I was missing a file in my panel library and I uninstalled and reinstalled multiple times and it still didn't show up.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df["Max Temp"] < 290) & (city_data_df["Max Temp"] >= 60)
                           & (city_data_df["Humidity"] < 90) & (city_data_df["Wind Speed"] < 130)]

# Drop any rows with null values
city_data_df.dropna(inplace=True)

# Display sample data
city_data_df.set_index('City_ID')


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,grytviken,-54.2811,-36.5092,272.96,88,95,5.43,GS,1731376041
2,coquimbo,-29.9533,-71.3436,287.03,69,0,3.09,CL,1731375777
3,kamaishi,39.2667,141.8833,286.43,52,87,1.74,JP,1731376045
5,polyarnyy,69.1989,33.4478,275.51,88,100,6.83,RU,1731376047
6,ushuaia,-54.8000,-68.3000,280.96,81,75,1.03,AR,1731376048
...,...,...,...,...,...,...,...,...,...
562,bolotnoye,55.6717,84.3981,262.65,89,100,7.19,RU,1731376744
564,sosnovo-ozerskoye,52.5249,111.5418,264.87,72,100,1.22,RU,1731376747
565,yuanping,38.7153,112.7575,283.82,47,99,0.65,CN,1731376748


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Lat", "Lng", "Humidity", "Country"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5,"Hotel Name","")

# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
0,grytviken,-54.2811,-36.5092,88,GS,
2,coquimbo,-29.9533,-71.3436,69,CL,
3,kamaishi,39.2667,141.8833,52,JP,
5,polyarnyy,69.1989,33.4478,88,RU,
6,ushuaia,-54.8000,-68.3000,81,AR,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
limit = 25
params = {
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
grytviken - nearest hotel: No hotel found
coquimbo - nearest hotel: No hotel found
kamaishi - nearest hotel: No hotel found
polyarnyy - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
zyryanskoye - nearest hotel: No hotel found
ofunato - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
ilulissat - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
shahr-e babak - nearest hotel: No hotel found
invercargill - nearest hotel: No hotel found
ancud - nearest hotel: No hotel found
anadyr - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
ketchikan - nearest hotel: No hotel found
nuuk - nearest hotel: No hotel found
aasiaat - nearest hotel: No hotel found
rawson - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
sitka - nearest hotel: No hotel found
badger - neare

,City,Lat,Lng,Humidity,Country,Hotel Name
0,grytviken,-54.2811,-36.5092,88,GS,No hotel found
2,coquimbo,-29.9533,-71.3436,69,CL,No hotel found
3,kamaishi,39.2667,141.8833,52,JP,No hotel found
5,polyarnyy,69.1989,33.4478,88,RU,No hotel found
6,ushuaia,-54.8000,-68.3000,81,AR,No hotel found
...,...,...,...,...,...,...
562,bolotnoye,55.6717,84.3981,89,RU,No hotel found
564,sosnovo-ozerskoye,52.5249,111.5418,72,RU,No hotel found
565,yuanping,38.7153,112.7575,47,CN,No hotel found
566,kargil,34.5667,76.1000,63,IN,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE
#I kept recieving an error that I was missing a file in my panel library and I uninstalled and reinstalled multiple times and it still didn't show up.

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)